In [2]:
# Because our images are fairly small, it will be faster to just cache them all in RAM rather than perform IO reads at runtime
import os
import matplotlib.pyplot as plt

LLabels = [] # All the labels, in order
LImages = [] # All the images, in order

for root, dirs, files in os.walk("plots"):
   for name in files:
      shortnameSplitted = os.path.basename(name).split("_")
      LLabels.append((shortnameSplitted[1],shortnameSplitted[2]))
      LImages.append(plt.imread(os.path.join(root, name)))
      plt.show()

print(len(LImages))
print(len(LLabels))

900
900
